In [1]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

import glob
import os

from scipy.stats import zscore

/home/jonah/.conda/envs/odc/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
weather_path = os.path.join('..','data','weather','precip_temp.csv')
weather_df = pd.read_csv(weather_path)
clients_path = os.path.join('..','data','wifi','**','Clients per day.csv')
clients_df = pd.concat(map(lambda csv: pd.read_csv(csv, parse_dates=[0]),
                           glob.glob(clients_path)), ignore_index=True)
weekday_labels = []
for d in range(7):
    label = 'weekday-%i' % d
    weekday_labels.append(label)
    clients_df[label] = [int(dt.weekday()==d) for dt in clients_df['Time']]
# Put clients date in the same format as it is in the weather data
clients_df['Time'] = [dt.strftime('%Y-%m-%d') for dt in clients_df['Time']]

# This merge ignores the missing data point in the weather data
all_data = clients_df.merge(weather_df, left_on='Time', right_on='DATE')

cols_to_norm = ['TMIN', 'TMAX', 'PRCP', '# Clients']
for col in cols_to_norm:
    all_data[col] = zscore(all_data[col])

x_labels = weekday_labels + ['PRCP', 'TMAX', 'TMIN']
y_labels = ['# Clients']
x = all_data[x_labels].values
y = all_data[y_labels].values
all_data

,Time,# Clients,weekday-0,weekday-1,weekday-2,weekday-3,weekday-4,weekday-5,weekday-6,DATE,PRCP,TMAX,TMIN
0,2017-07-01,0.429791,0,0,0,0,0,1,0,2017-07-01,-0.343698,1.335709,1.295709
1,2017-07-02,0.039302,0,0,0,0,0,0,1,2017-07-02,-0.343698,1.006285,1.448834
2,2017-07-03,-0.035076,1,0,0,0,0,0,0,2017-07-03,-0.343698,1.241588,1.236814
3,2017-07-04,-0.537133,0,1,0,0,0,0,0,2017-07-04,-0.318790,1.112171,1.331045
4,2017-07-05,-0.165239,0,0,1,0,0,0,0,2017-07-05,0.328838,0.570976,1.201478
5,2017-07-06,1.136388,0,0,0,1,0,0,0,2017-07-06,-0.343698,1.029816,1.142583
6,2017-07-07,1.619850,0,0,0,0,1,0,0,2017-07-07,-0.343698,1.206292,1.236814
7,2017-07-08,0.150870,0,0,0,0,0,1,0,2017-07-08,-0.343698,1.135702,1.225036
8,2017-07-09,-0.871837,0,0,0,0,0,0,1,2017-07-09,-0.343698,0.982755,0.965900
9,2017-07-10,-1.001999,1,0,0,0,0,0,0,2017-07-10,-0.343698,1.300413,1.095468


In [20]:
epochs = 1000

model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(10,)))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))

model.summary()

model.compile(loss='mean_absolute_percentage_error',
              optimizer='Adam')

early_stopping = EarlyStopping(patience=50, verbose=1)
history = model.fit(x, y,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 50)                550       
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 51        
Total params: 3,151
Trainable params: 3,151
Non-trainable params: 0
_________________________________________________________________
Train on 316 samples, validate on 36 samples
Epoch 1/1000
316/316 [==============================] - 1s 4ms/step - loss: 238.2473 - val_loss

316/316 [==============================] - 0s 319us/step - loss: 99.9050 - val_loss: 100.9051
Epoch 68/1000
316/316 [==============================] - 0s 330us/step - loss: 116.6218 - val_loss: 99.8109
Epoch 69/1000
316/316 [==============================] - 0s 327us/step - loss: 136.6406 - val_loss: 99.8242
Epoch 70/1000
316/316 [==============================] - 0s 342us/step - loss: 105.4950 - val_loss: 99.8156
Epoch 71/1000
316/316 [==============================] - 0s 357us/step - loss: 122.3985 - val_loss: 99.1840
Epoch 72/1000
316/316 [==============================] - 0s 455us/step - loss: 118.8654 - val_loss: 97.8617
Epoch 73/1000
316/316 [==============================] - 0s 591us/step - loss: 107.0412 - val_loss: 97.7278
Epoch 74/1000
316/316 [==============================] - 0s 359us/step - loss: 103.6889 - val_loss: 98.0172
Epoch 75/1000
316/316 [==============================] - 0s 364us/step - loss: 111.1941 - val_loss: 98.6891
Epoch 76/1000
316/316 [===================